<a href="https://colab.research.google.com/github/bksaini078/MachineLearningPractice/blob/main/BiLSTM_Attn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Bhupender Kumar Saini



# Declaring Libraries 





In [ ]:
#please install contractions 
import numpy as np
import re
import nltk
# import contractions
import string
import pandas as pd
import os
import numpy as np
import random
import spacy
import time
import re
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot


from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim import utils 
from gensim.test.utils import get_tmpfile
from nltk.corpus import stopwords
import nltk
import en_core_web_sm
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
from pathlib import Path

# nlp = en_core_web_sm.load()
# nltk.download('stopwords')
# nltk.download('punkt')
# porter=PorterStemmer()


In [ ]:
import tensorflow as tf 

import tensorflow.keras as tfk
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda
from tensorflow.keras.models import Model
import numpy as np
import datetime
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras import regularizers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
#this is to enable eager execution
tf.compat.v1.enable_eager_execution()
path = '/content/drive/MyDrive/Semi_super/ExperimentFolds/fakehealth/preprocessed/'


# Tokenization

In [ ]:
full_article_temp=[]
for i in range(5):
    # print('-------------FOLD Number:---------------',i)
    x_train= np.load(path+'train_'+str(i)+'_x.npy', allow_pickle=True)
    x_val= np.load(path+'dev_'+str(i)+'_x.npy', allow_pickle=True)
    x_test= np.load(path+'test_x.npy', allow_pickle=True)
    full_article_temp= np.hstack((x_train, x_val, full_article_temp, x_test))

def tokenization(x_train, x_val,  x_test, x_unlabel, maxlen ):

    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing import sequence
    import numpy as np
    tokenizer = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ',
                          char_level=False, oov_token=None, document_count=0)
    # full_article = np.hstack((x_train, x_test, x_unlabel))
    full_article= np.hstack(( x_unlabel, full_article_temp))
    tokenizer.fit_on_texts(full_article)
    x_train_token = tokenizer.texts_to_sequences(x_train)
    x_test_token = tokenizer.texts_to_sequences(x_test)
    x_val_token = tokenizer.texts_to_sequences(x_val)
    x_unlabel_token = tokenizer.texts_to_sequences(x_unlabel)

    x_train_seq = sequence.pad_sequences(x_train_token, maxlen=maxlen,padding='post')
    x_test_seq = sequence.pad_sequences(x_test_token, maxlen=maxlen,padding='post')
    x_val_seq = sequence.pad_sequences(x_val_token, maxlen=maxlen,padding='post')
    x_unlabel_tar= sequence.pad_sequences(x_unlabel_token, maxlen=maxlen,padding='post')
    # defining vocalbury size
    vocab_size = len(tokenizer.word_index) + 1


    x_train = x_train_seq
    x_test = x_test_seq
    return x_train,x_val_seq, x_test, x_unlabel_tar, vocab_size, tokenizer

# Noise Creator 

In [ ]:
def instant_noise(x_train, y_train, x_unlabel, n_ratio ):
    '''this function introduce noise in the training data for mean teacher model , 
    this function is used in calculating classification cost, user have to provide 
    amount of noise, want to add(ratio) in train data and test train split ratio too'''
    #amount of noise need to add in x_train data 
    # print('In noise function:e', np.shape(x_train))
    noise=int(np.shape(x_train)[0]*n_ratio)
 
    # taking column of x_train, need it later 
    x_column = np.shape(x_train)[1]

    if noise <= int(np.shape(x_unlabel)[0]):

        #taking number of noise from unlabel data 
        ratio_noise = x_unlabel[:noise]

        # creating -1 label for noise data 
        y_unlabel=np.full((np.shape(ratio_noise)[0], 1), -1)

        # adding noise in train data 
        x = np.append(x_train, ratio_noise, axis=0)
        # print(np.shape(x))
        y = np.append(y_train, y_unlabel, axis=0)
        x = np.append(x,y, axis=1)
        x= x[noise:]
        row = np.shape(x)[0]
        # print('In noise function:l', np.shape(x_train))

        # shufflin data 
        x =np.random.permutation(x)
        # print(np.shape(x))
        
        #seperating label from x 
        y_train_n=np.reshape(x[:,x_column],(row,1))
        x_train_n=x[0:len(x),0:x_column]
        # y_train_n= np.reshape(y[:len(x),0],(train_split,1))

        
    else :
        print('error: Insufficient unlabel data available !')

    return x_train_n, y_train_n


# Model Declaration



In [ ]:
def BiLstmModel_attention(maxlen, vocab_size):
  import keras
  tf.keras.backend.clear_session()
  inputs = keras.Input(shape=(maxlen,))
  x = Embedding(vocab_size, 128, input_length=None)(inputs)
  x = Bidirectional(LSTM(128,return_sequences= True))(x)
  x = tf.keras.layers.Attention()([x,x])
  x= Flatten()(x)
  x = Dense(64)(x)
  x = Dense(1, activation='sigmoid')(x)
  return Model(inputs,x)



# Evaluation Functions

In [ ]:

#Function to create confusion matrix 
def Confusion_matrix(model,x_test,y_true, threshold, caption='Confusion matrix'):
    '''this function will create confusion matrix with predicted value and true label'''
    y_hat= model.predict(x_test)
    y_pred=(np.greater_equal(y_hat,threshold)).astype(int)
    cm=confusion_matrix(y_true,y_pred)
    # print(cm)
    # calculating recall , precision and f1 score 
    tp_and_fp=np.sum(cm[:,1])
    tn_and_fp=np.sum(cm[0,:])
    tp_and_fn = np.sum(cm[1, : ])
    tp_and_tn= np.trace(cm)
    tp=(tp_and_fp-tn_and_fp+tp_and_tn)/2
    '''handling with divide by zero is pending'''
    #TODO: handling of divide by zero 
    precision=tp/tp_and_fp 
    recall = tp/tp_and_fn
    accuracy= np.trace(cm)/np.sum(cm)
    # f1_score=sklearn.metrics.f1_score(y_true, y_pred)
    f1_score= (2*precision*recall)/(precision+recall)
    print('Precision:', precision)
    print('Recall:', recall)
    print('f1 Score:', f1_score)
    print('Accuracy:', accuracy)

    # plt.show()
    return cm, accuracy, precision, recall, f1_score
def prec_rec_f1score(y_true,x_test,model):
    from sklearn.metrics import precision_recall_fscore_support
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    from sklearn import metrics
    bce = tf.keras.losses.BinaryCrossentropy()
    y_hat= model.predict(x_test)
    y_pred=(np.greater_equal(y_hat,0.51)).astype(int)

    pr_re_f1score_perclass= precision_recall_fscore_support(y_true, y_pred, average=None)
    pr_re_f1score_average=precision_recall_fscore_support(y_true, y_pred, average='micro')
    precision=precision_score(y_true,y_pred,average=None)
    recall = recall_score(y_true,y_pred,average=None)
    accuracy= accuracy_score(y_true,y_pred)
    f1_score=f1_score(y_true,y_pred)
    #per class
    precision_true=pr_re_f1score_perclass[0][1]
    precision_fake=pr_re_f1score_perclass[0][0]
    recall_true=pr_re_f1score_perclass[1][1]
    recall_fake=pr_re_f1score_perclass[1][0]
    f1score_true= pr_re_f1score_perclass[2][1]
    f1score_fake= pr_re_f1score_perclass[2][0]

    # AUC
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_hat, pos_label=None)
    AUC= metrics.auc(fpr, tpr)

    metrices_name=['accuracy','precision_true','precision_fake','recall_true','recall_fake','f1score_true','f1score_fake', 'AUC']
    metrices_value=[accuracy, precision_true, precision_fake, recall_true, recall_fake, f1score_true, f1score_fake, AUC]
    i=0
    for item in metrices_name:
        print(item +':' ,metrices_value[i])
        i+=1

    return accuracy, precision_true, precision_fake, recall_true, recall_fake, f1score_true, f1score_fake, AUC 


In [ ]:
import matplotlib.pyplot as plt
def scatter_plot(logits, y_t, title):
    marker_size=20
    figure = plt.figure(figsize=(20, 6))
    plt.scatter(logits,logits, marker_size, c=y_t)
    plt.title(title)
    plt.xlabel("Predicted Probability")
    plt.ylabel("Predicted Probability")
    cbar= plt.colorbar()
    cbar.set_label("Probability", labelpad=+1)
    plt.show()
    return

# accuracy 
def model_evaluation(model, x_test, y_true, name):
    y_hat= model(x_test)
    y_pred=(np.greater(y_hat,0.505)).astype(int)
    cm=confusion_matrix(y_true,y_pred)
    
    accuracy= np.trace(cm)/np.sum(cm)
    
    print(name+ ":")
  
    print(' Accuracy:', accuracy)

    # this will plot the result 
    # scatter_plot(y_hat,y_true, title=name)

    return accuracy #precision, recall, f1_score, accuracy
def plot_roc(fpr,tpr,label):
    #  plot the roc curve for the model
    pyplot.plot(fpr, tpr, linestyle='--', label=label)

    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    # show the plot 
    pyplot.show()
    return

# Cost Functions

In [ ]:
# declaring loss function
# ref:https://github.com/CuriousAI/mean-teacher/tree/master/tensorflow/mean_teacher  updated according to our need .
def classification_costs(logits, labels):
    """ Commputing classification cost , after removing labels -1 of unlabelled data and then calculating 
    the binary cross entropy .
    """
    applicable = tf.not_equal(labels, -1)

     # Change -1s to zeros to make cross-entropy computable
    labels = tf.where(applicable, labels, tf.zeros_like(labels))

    # This will now have incorrect values for unlabeled examples
    per_sample = tf.keras.losses.binary_crossentropy(labels,logits)
    # Retain costs only for labeled
    per_sample = tf.where(applicable, per_sample, tf.zeros_like(per_sample))
    # Take mean over all examples, not just labeled examples.
   
    loss = tf.math.divide( tf.reduce_mean(tf.reduce_sum(per_sample)), np.shape(per_sample)[0])

    return loss

#custom loss function
def Overall_Cost(classification_cost, consistency_cost, ratio=0.5):
    return (ratio * classification_cost) + ((1 - ratio)*consistency_cost)
#function for consistency cost 
def Consistency_Cost(teacher_output, student_output):
    #Kl divergence 
    # kl = tf.keras.losses.KLDivergence()
    # sq_diff_layer=kl(teacher_output, student_output).numpy()
    
    #MSE
    sq_diff_layer = tf.reduce_mean(tf.math.squared_difference(teacher_output, student_output))
    return sq_diff_layer
def ema(student_model, teacher_model, alpha):
    '''
    Calculates the exponential moving average of the student model weights and updates the teacher model weights
    formula:
    t_i = alpha * t_{i-1} + (1 - alpha) * s_i, with default alpha = 0.99
    t_i = weights of teacher model in current epoch
    s_i = weights of student model in current epoch
    '''
    #taking weights 
    student_weights = student_model.get_weights()
    teacher_weights = teacher_model.get_weights()

    #length must be equal otherwise it will not work 
    assert len(student_weights) == len(teacher_weights), 'length of student and teachers weights are not equal Please check. \n Student: {}, \n Teacher:{}'.format(
        len(student_weights), len(teacher_weights))

    new_layers = []
    for i, layers in enumerate(student_weights):
        new_layer = alpha*(teacher_weights[i]) + (1-alpha)*layers
        new_layers.append(new_layer)
    teacher_model.set_weights(new_layers)
    return teacher_model


# Writing Report

In [ ]:
def report_writing(Model,lr,Batch_Size, Epoch,Alpha,Ratio, train_accuracy,test_accuracy,precision_true,precision_fake,recall_true,recall_fake,f1score_true,f1score_fake,AUC,comment):
    x = datetime.datetime.now()
    report_df = pd.DataFrame(columns=['Date', 'Model','Learning Rate','Batch_Size', 'Epoch','Alpha','Ratio','Train_Accuracy',
                                      'Test_Accuracy', 'Precision_True','Precision_Fake','Recall_True','Recall_Fake','F1_Score_True','F1_Score_Fake','AUC',
                                      'comment'])
    report_df = report_df.append({'Date' : x.strftime("%c"), 'Model' :Model,'Learning Rate':lr,'Batch_Size' : Batch_Size, 'Epoch': Epoch,'Alpha': Alpha,'Ratio': Ratio,'Train_Accuracy': train_accuracy,
                                  'Test_Accuracy': test_accuracy, 'Precision_True': precision_true,'Precision_Fake': precision_fake,'Recall_True': recall_true,'Recall_Fake': recall_fake,'F1_Score_True': f1score_true,'F1_Score_Fake': f1score_fake, 'AUC':AUC,'comment': comment}, ignore_index=True)
    my_file = Path(path+report_name)

    if my_file.exists():
        report_df.to_csv(path+report_name,mode='a', header= False , index = False)
    else:
        report_df.to_csv(path+report_name,mode='w', header= True , index= False) 
    return 


# Supervised Model

In [ ]:


def train_supervised(epochs, batch_size, lr, x_train, y_train, x_val, y_val,x_test, y_test,maxlen,vocab_size):

    model_supervised = BiLstmModel_attention(maxlen, vocab_size)
    model_supervised.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= lr ),loss= 'binary_crossentropy', metrics=['accuracy'])
    print('Training supervised Model...')
    history=model_supervised.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_val,y_val))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()  

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    # evaluation
    train_accuracy=history.history['accuracy'][len(history.epoch)-1]
       
    test_accuracy,precision_true,precision_fake,recall_true,recall_fake,f1score_true,f1score_fake,AUC = prec_rec_f1score(y_test,x_test,model_supervised)
   
    report_writing('BILSTM_attn',lr, batch_size,len(history.epoch),'NaN','NaN', train_accuracy, 
                   test_accuracy, precision_true, precision_fake, recall_true, recall_fake, 
                   f1score_true, f1score_fake,AUC,'Supervised Learning_attention')    


# MEAN teacher
In this updation takes place during each step/batch. This model doesnt work 

In [ ]:
def train_MeanTeacher(epochs, batch_size, alpha, lr, ratio,x_train, y_train,x_val,y_val, x_test, y_test, x_unlabel_tar,vocab_size,maxlen):

    #preparing the training dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    
    #preparing the target dataset 
    tar_dataset =  tf.data.Dataset.from_tensor_slices(x_unlabel_tar)
    tar_dataset = tar_dataset.shuffle(buffer_size=1024).batch(batch_size)

    #declaring optimiser
    optimizer= tf.keras.optimizers.Adam(learning_rate= lr ) #trying changing learning rate , sometimes it gives good result 
    train_metrics = tf.keras.metrics.BinaryAccuracy(name='Binary_Accuracy')
    val_acc_metric = tf.keras.metrics.BinaryAccuracy(name="Binary_Acc")
    teacher_acc_metric = tf.keras.metrics.BinaryAccuracy(name="Binary_Acc_teacher") 
    # Creating model
    student = BiLstmModel_attention(maxlen, vocab_size)
    student.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= lr ),loss= 'binary_crossentropy', metrics=['accuracy'])
    teacher = BiLstmModel_attention(maxlen, vocab_size)
    teacher.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= lr ),loss= 'binary_crossentropy', metrics=['accuracy'])


    # collecting costs
  
    train_accuracy=[]
    steps=[]

    #training teacher with one epoch 
   
    #this I am doing to get all steps details in epoch
    i=0
    print('Train Mean teacher Model...')
    # teacher.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    teacher.fit(x_train,y_train, batch_size=batch_size, epochs=1)

    acc_t=0
    #false positive rate and true positive rate
    fpr=[]
    tpr=[]
    # x_unlabel_tar= tf.convert_to_tensor(x_unlabel_tar)
    for epoch in range(1,epochs+1):  
        print(*"*****************")
        print('Start of epoch %d' % (epoch,))
        print(*"*****************")
        #iteration over batches 
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
             with tf.GradientTape() as tape:
         
                # adding instant noise
                iterator_unlabel = iter(tar_dataset)
                x_batch_unlabel = iterator_unlabel.get_next()

                '''this is one method of adding -1 label using unlable data'''
                x_train_n,y_train_n= instant_noise(x_batch_train,y_batch_train,x_batch_unlabel,0.2)

                # Run the forward pass of the layer
                logits= student(x_train_n, training= True)  
                # logits_acc =  student(x_batch_sn, training= False) 

                # doing this because then accuracy cannot be calculated
                logits_acc= student(x_batch_train, training= True) 

                # calculating accuracy
                train_metrics(y_batch_train,logits_acc)  

                #Calculating classification cost 
                classification_cost = classification_costs(logits,y_train_n)
                # classification.append(classification_cost)
         
                x_train_n1,_ = instant_noise(x_batch_train,y_batch_train,x_batch_unlabel,0.2)
                           
                tar_teacher = teacher(x_train_n1) #x_batch_train
                #  tar_student= student(x_train_n)
                consistency_cost= Consistency_Cost(tar_teacher,logits) 
                # consistency.append(consistency_cost)

                overall_cost= Overall_Cost(classification_cost, consistency_cost, ratio=0.5)
 
                #adding loss to student model 
             grads= tape.gradient(overall_cost, student.trainable_weights)
             i=i+1
             steps.append(i)
   
             # the value of the variables to minimize the loss.
             optimizer.apply_gradients(zip(grads, student.trainable_weights))
             teacher= ema(student, teacher, alpha=alpha)

        train_acc = train_metrics.result()
        print(alpha)
   
        #appending training accuracy
        train_accuracy.append(train_acc)

       
            # Reset training metrics at the end of each epoch
        train_metrics.reset_states()
   
        # Run a validation loop at the end of each epoch.
        print('*******STUDENT*************')
        prec_rec_f1score(y_val,x_val,student)
        print('*******TEACHER*************')
        prec_rec_f1score(y_val,x_val,teacher)

   
        if epoch >= 10 and epoch % 5 ==0 :
            print('---------------------------STUDENT--------------------------')
            test_accuracy,precision_true,precision_fake,recall_true,recall_fake,f1score_true,f1score_fake,AUC = prec_rec_f1score(y_test,x_test,student)
            report_writing('Student',lr, batch_size,epoch,alpha,ratio, train_acc.numpy(), 
                           test_accuracy, precision_true, precision_fake, recall_true, recall_fake, 
                           f1score_true, f1score_fake,AUC,'MeanTeacher-Attention')  
            print('-----------------------------------------------------------------')
    
            print('---------------------------TEACHER---------------------------------')
       
            test_accuracy,precision_true,precision_fake,recall_true,recall_fake,f1score_true,f1score_fake,AUC = prec_rec_f1score(y_test,x_test,teacher)
            report_writing('Teacher',lr, batch_size,epoch,alpha,ratio, train_acc.numpy(), 
                           test_accuracy, precision_true, precision_fake, recall_true, recall_fake, 
                           f1score_true, f1score_fake,AUC,'MeanTeacher-Attention') 
       
            print('-----------------------------------------------------------------')
        # if epoch >= 10 and epoch% 5==0 :
        #     teacher.save(path+'/model/MT_unlabel_'+str(epoch))    

    tf.keras.backend.clear_session()
    return teacher

    

# Main Functions

# Hyper Parameters

In [ ]:
tf.keras.backend.clear_session()


lr=0.0001

batch_size= 64
#for mean teacher 
ratio =0.5
alpha=0.99 #(0.90-0.99)
maxlen=510

# BiLSTM-Attn

In [ ]:

# report_name='report/BiLSTM-Attn.csv'
# for i in range(5):
#     x_train= np.load(path+'train_'+str(i)+'_x.npy', allow_pickle=True)
#     y_train= np.load(path+'train_'+str(i)+'_y.npy', allow_pickle=True)

#     x_val= np.load(path+'dev_'+str(i)+'_x.npy', allow_pickle=True)
#     y_val= np.load(path+'dev_'+str(i)+'_y.npy', allow_pickle=True)

#     x_test= np.load(path+'test_x.npy', allow_pickle=True)
#     y_test= np.load(path+'test_y.npy', allow_pickle=True)

#     print('train data size:', np.shape(x_train))
#     print('train data True/Fake count:', np.count_nonzero(y_train), len(y_train)- np.count_nonzero(y_train))
#     print('val data size:', np.shape(x_val))
#     print('val data True/Fake count:', np.count_nonzero(y_val), len(y_val)- np.count_nonzero(y_val))

#     print('test data size:', np.shape(x_test))
#     print('test data True/Fake count:', np.count_nonzero(y_test), len(y_test)- np.count_nonzero(y_test))

#     x_unlabel= np.load(path+'unlabeled_'+'all'+'_x.npy', allow_pickle=True)
#     x_train,x_val, x_test, x_unlabel, vocab_size, tokenizer= tokenization(x_train ,x_val,  x_test, x_unlabel, maxlen)
#     print(vocab_size)

#     epochs=10
#     model = train_supervised(epochs, batch_size, lr, x_train, y_train, x_val, y_val,x_test, y_test,maxlen,vocab_size)
    



# BiLSTM-Attn-MT-Mix

In [ ]:
report_name='report/BiLSTM-Attn-MT-Mix.csv'
for i in range(1):
    print('-------------FOLD Number:---------------',i)
    x_train= np.load(path+'train_'+str(i)+'_x.npy', allow_pickle=True)
    y_train= np.load(path+'train_'+str(i)+'_y.npy', allow_pickle=True)

    x_val= np.load(path+'dev_'+str(i)+'_x.npy', allow_pickle=True)
    y_val= np.load(path+'dev_'+str(i)+'_y.npy', allow_pickle=True)

    x_test= np.load(path+'test_x.npy', allow_pickle=True)
    y_test= np.load(path+'test_y.npy', allow_pickle=True)
    x_unlabel= np.load(path+'unlabeled_'+'mix'+'_x.npy', allow_pickle=True)

    print('train data size:', np.shape(x_train))
    print('train data True/Fake count:', np.count_nonzero(y_train), len(y_train)- np.count_nonzero(y_train))
    print('val data size:', np.shape(x_val))

    print('test data size:', np.shape(x_test))
    print('train data True/Fake count:', np.count_nonzero(y_test), len(y_test)- np.count_nonzero(y_test))

    x_train,x_val, x_test, x_unlabel, vocab_size, tokenizer= tokenization(x_train ,x_val,  x_test, x_unlabel, maxlen)
    epochs=20
    model = train_MeanTeacher(epochs, batch_size, alpha, lr, ratio,x_train, y_train,x_val,y_val, x_test, y_test, x_unlabel,vocab_size,maxlen)

-------------FOLD Number:--------------- 0
train data size: (1256,)
train data True/Fake count: 377 879
val data size: (316,)
test data size: (394,)
train data True/Fake count: 158 236
Train Mean teacher Model...
20/20 [==============================] - 60s 3s/step - loss: 0.6395 - accuracy: 0.6712
* * * * * * * * * * * * * * * * *
Start of epoch 1
* * * * * * * * * * * * * * * * *
0.99
*******STUDENT*************


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.6487341772151899
precision_true: 0.0
precision_fake: 0.6487341772151899
recall_true: 0.0
recall_fake: 1.0
f1score_true: 0.0
f1score_fake: 0.7869481765834934
AUC: 0.4864864864864865
*******TEACHER*************
accuracy: 0.6487341772151899
precision_true: 0.0
precision_fake: 0.6487341772151899
recall_true: 0.0
recall_fake: 1.0
f1score_true: 0.0
f1score_fake: 0.7869481765834934
AUC: 0.5809712151175566
* * * * * * * * * * * * * * * * *
Start of epoch 2
* * * * * * * * * * * * * * * * *
0.99
*******STUDENT*************
accuracy: 0.6487341772151899
precision_true: 0.0
precision_fake: 0.6487341772151899
recall_true: 0.0
recall_fake: 1.0
f1score_true: 0.0
f1score_fake: 0.7869481765834934
AUC: 0.6735223027905954
*******TEACHER*************
accuracy: 0.6487341772151899
precision_true: 0.0
precision_fake: 0.6487341772151899
recall_true: 0.0
recall_fake: 1.0
f1score_true: 0.0
f1score_fake: 0.7869481765834934
AUC: 0.5027905954735223
* * * * * * * * * * * * * * * * *
Start of epoch 3
* *

# BiLSTM-Attn-MT-All

In [ ]:
report_name='report/BiLSTM-Attn-MT-All.csv'
for i in range(5):
    x_train= np.load(path+'train_'+str(i)+'_x.npy', allow_pickle=True)
    y_train= np.load(path+'train_'+str(i)+'_y.npy', allow_pickle=True)

    x_val= np.load(path+'dev_'+str(i)+'_x.npy', allow_pickle=True)
    y_val= np.load(path+'dev_'+str(i)+'_y.npy', allow_pickle=True)

    x_test= np.load(path+'test_x.npy', allow_pickle=True)
    y_test= np.load(path+'test_y.npy', allow_pickle=True)


    x_unlabel= np.load(path+'unlabeled_'+'all'+'_x.npy', allow_pickle=True)
    print('train data size:', np.shape(x_train))
    print('train data True/Fake count:', np.count_nonzero(y_train), len(y_train)- np.count_nonzero(y_train))
    print('val data size:', np.shape(x_val))

    print('test data size:', np.shape(x_test))
    print('train data True/Fake count:', np.count_nonzero(y_test), len(y_test)- np.count_nonzero(y_test))

    x_train,x_val, x_test, x_unlabel, vocab_size, tokenizer= tokenization(x_train ,x_val,  x_test, x_unlabel, maxlen)

    epochs=15
    model = train_MeanTeacher(epochs, batch_size, alpha, lr, ratio,x_train, y_train,x_val,y_val, x_test, y_test, x_unlabel,vocab_size,maxlen)